In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options , service=Service(ChromeDriverManager().install()))
driver.get('https://www.onmap.co.il/')
search = driver.find_element(By.NAME, 'SearchInput')
search.send_keys("חיפה")
search_box = driver.find_element(By.CLASS_NAME, 'btn-search')
search_box.send_keys(Keys.RETURN)

In [3]:
try:
    viewContainer = WebDriverWait(driver , 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "viewContainer"))
    )
    print(viewContainer.page_source)
except:
    driver.quit()
